In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/properatti.csv')
data.head(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [2]:
''' Vemos las columnas '''

data.loc[10,:]


Unnamed: 0                                                                   10
operation                                                                  sell
property_type                                                             house
place_name                                                              Córdoba
place_with_parent_names                             |Argentina|Córdoba|Córdoba|
country_name                                                          Argentina
state_name                                                              Córdoba
geonames_id                                                         3.86026e+06
lat-lon                                                 -31.4200833,-64.1887761
lat                                                                    -31.4201
lon                                                                    -64.1888
price                                                                     70000
currency                                

In [3]:
''' vemos los nulos '''

data['price_aprox_usd'].isna().sum()


20410

In [4]:
''' Exploramos las columnas de los precios para ver si hay datos no nulos'''

data.loc[data['price_aprox_usd'].isna()].loc[:,['price','price_aprox_local_currency','price_usd_per_m2','price_per_m2']]




,price,price_aprox_local_currency,price_usd_per_m2,price_per_m2
5,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN
41,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN
67,NaN,NaN,NaN,NaN
...,...,...,...,...
121210,NaN,NaN,NaN,NaN
121211,NaN,NaN,NaN,NaN
121212,NaN,NaN,NaN,NaN
121213,NaN,NaN,NaN,NaN


In [5]:
''' quitar los nulos precios '''

data = data.drop(data[data['price_aprox_usd'].isna()].index)
cant_nulls = data.price_aprox_usd.isnull().sum()
print(cant_nulls)



0


In [6]:
'''place names nulls'''
data.place_name.isnull().sum()


23

In [7]:
''' completar los places names '''
data.loc[data.place_name.isnull(),'place_name'] = data.loc[data.place_name.isnull()].place_with_parent_names.str.split('|', expand=True)[3]

'''place names nulls despues de completar'''

data.place_name.isnull().sum()


0

In [8]:

''' rooms antes limpiar'''
cant_nulls = data.rooms.isnull().sum()
print(cant_nulls)

61231


In [9]:
''' Leo los ambientes para completar la info'''

import re

patron_ambientes = re.compile("(?P<Ambientes>\d+)(?P<tipo>amb|\samb)",flags = re.IGNORECASE)
patron_habitaciones = re.compile("(?P<Habitaciones>\d+)(?P<tipo>hab|\shab|dorm|\sdorm)",flags = re.IGNORECASE)

def extractRooms(row):
    ambientes_match = None
    if ( row['description'] and isinstance(row['description'],str) ):
        ambientes_match = patron_ambientes.search(row['description'])
    if ((ambientes_match is None) and row['title'] and isinstance(row['title'],str)):
        ambientes_match = patron_ambientes.search(row['title'])
    if (ambientes_match is not None):
        return int(ambientes_match.groups()[0]) 
    else: 
        if (row['description'] and isinstance(row['description'],str)):
            ambientes_match = patron_habitaciones.search(row['description'])
        if ((ambientes_match is None) and row['title'] and isinstance(row['title'],str)):
            ambientes_match = patron_habitaciones.search(row['title'])
        if (ambientes_match is not None):
            return int(ambientes_match.groups()[0])+1
        return None
        
        
data.loc[data.rooms.isnull(),'rooms'] = data.loc[data.rooms.isnull()].apply(extractRooms,axis=1)

''' rooms despues de limpiar'''

data.rooms.isnull().sum()


24521

In [10]:
''' nulos en superficie'''

data.surface_total_in_m2.isnull().sum()


31944

In [11]:

patron_meters = re.compile("(?P<Metros>\d+)(?P<tipo>metro|\smetro|mts|\smts|m2|\sm2)",flags = re.IGNORECASE)


def extractMeters(row):
    meters_match = None
    if ( row['description'] and isinstance(row['description'],str) ):
        meters_match = patron_meters.search(row['description'])
    if ((meters_match is None) and row['title'] and isinstance(row['title'],str)):
        meters_match = patron_meters.search(row['title'])
    if (meters_match is not None):
        return meters_match.groups()[0]
    return None

data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull()].apply(extractMeters,axis=1)

data.loc[data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull()].apply(lambda x: x['surface_covered_in_m2'],axis=1)




In [12]:
data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull()].surface_total_in_m2.fillna(value=np.nan)
data.loc[:,'surface_total_in_m2'] = data.loc[:,'surface_total_in_m2'].astype("float")

''' cuantos quedaron'''

data.surface_total_in_m2.isnull().sum()


5904

In [13]:
''' cuantos departamentos tienen el piso en null '''
len(data.loc[(data.property_type == 'apartment') & (data.floor.isnull())].index)


53700

In [37]:
patron_floor = re.compile("(?P<piso0>planta baja|PB)|(?P<piso1>1er piso|primer piso|piso 1|1º piso|piso 1º)|(?P<piso2>2do piso|segundo piso|piso 2|2º piso|piso 2º)|(?P<piso3>3er piso|tercer piso|piso 3|3º piso|piso 3º)|(?P<piso4>4to piso|cuarto piso|piso 4|4º piso|piso 4º)|(?P<piso5>5to piso|quinto piso|piso 5|5º piso|piso 5º)|(?P<piso6>6to piso|sexto piso|piso 6|6º piso|piso 6º)|(?P<piso7>7to piso|séptimo piso|piso 7|7º piso|piso 7º)|(?P<piso8>8vo piso|octavo piso|piso 8|8º piso|piso 8º)|(?P<piso9>9no piso|noveno piso|piso 9|9º piso|piso 9º)|(?P<piso10>10mo piso|décimo piso|piso 10|10º piso|piso 10º)|(?P<piso11>11vo piso|onceavo piso|piso 11|11º piso|piso 11º)|(?P<piso12>12vo piso|doceavo piso|piso 12|12º piso|piso 12º)|(?P<piso13>13vo piso|treceavo piso|piso 13|13º piso|piso 13º)|(?P<piso14>14vo piso|cartoceavo piso|piso 14|14º piso|piso 14º)|(?P<piso15>15vo piso|quinceavo piso|piso 15|15º piso|piso 15º)",flags = re.IGNORECASE)

def extractFloor(row):
    match = None
    if (row['description'] and isinstance(row['description'],str) ):
        match = patron_floor.match(row['description'])
    if ( (match is None) and row['title'] and isinstance(row['title'],str) ):
        match = patron_floor.match(row['title'])
    if (match is not None):    
        return int(match.lastgroup.replace('piso',''))               
    return row['floor']

data.loc[(data.property_type == 'apartment') & (data.floor.isnull()),'floor'] = data.loc[(data.property_type == 'apartment') & (data.floor.isnull())].apply(lambda x: extractFloor(x),axis =1)

''' cuantos departamentos tienen el piso en null '''
len(data.loc[(data.property_type == 'apartment') & (data.floor.isnull())].index)

53413

In [38]:
data.state_name.value_counts()


Capital Federal                 29018
Bs.As. G.B.A. Zona Norte        24058
Bs.As. G.B.A. Zona Sur          12085
Buenos Aires Costa Atlántica     9059
Córdoba                          9038
Bs.As. G.B.A. Zona Oeste         7333
Santa Fe                         5413
Buenos Aires Interior            1814
Mendoza                           476
Corrientes                        470
Río Negro                         361
Misiones                          348
Neuquén                           324
Tucumán                           213
San Luis                          208
Chubut                            164
Entre Ríos                        130
Salta                             102
La Pampa                           36
Chaco                              36
Tierra Del Fuego                   31
Formosa                            29
Catamarca                          26
Santa Cruz                         19
Jujuy                              10
San Juan                            4
Santiago Del

In [39]:

'''filtramos lugares con 5000 propiedades en adelante '''
data = data.groupby("state_name").filter(lambda x: len(x) > 5000)


In [40]:
''' columna a estrenar '''
patron_a_estrenar = re.compile("(?P<tipo>estrenar)",flags = re.IGNORECASE)

def aestrenar(row):
    if ( row['description'] and isinstance(row['description'],str) ):
        if (patron_a_estrenar.match(row['description'])):
                return True
    if ( row['title'] and isinstance(row['title'],str) ):
        if (patron_a_estrenar.match(row['title'])):
            return True
    return False

data['aAstrenar'] = data.apply(lambda x:aestrenar(x),axis = 1);

 


In [41]:
data['aAstrenar'].value_counts()

False    95995
True         9
Name: aAstrenar, dtype: int64

In [43]:
''' columna disposicion del dpto'''

patron_disposicion = re.compile("(?P<tipo>frente|lateral|contrafrente)",flags = re.IGNORECASE)

def disposicion(row):
    disposicion_match = None
    if ( row['description'] and isinstance(row['description'],str) ): 
        disposicion_match = patron_disposicion.search(row['description'])
    if ((patron_disposicion is None) and row['title'] and isinstance(row['title'],str)):
        disposicion_match = patron_disposicion.search(row['title'])
    if (disposicion_match is not None):
        return disposicion_match.groups()[0].lower()
    return None  

data['disposicion'] =  data.apply(lambda x:disposicion(x),axis = 1)
data['disposicion'].value_counts()


frente          27795
contrafrente     5185
lateral          1767
Name: disposicion, dtype: int64

In [44]:
data.price_aprox_usd.describe()


count    9.600400e+04
mean     2.411288e+05
std      3.854264e+05
min      0.000000e+00
25%      9.000000e+04
50%      1.450000e+05
75%      2.700000e+05
max      4.654544e+07
Name: price_aprox_usd, dtype: float64

In [ ]:
data.loc[data.price_aprox_usd > 8000000,['state_name','currency','price','price_aprox_usd','property_type','price_usd_per_m2','surface_total_in_m2','title','description']]

In [ ]:
data.loc[data.state_name == 'Capital Federal'].groupby(['place_name','property_type']).price_aprox_usd.describe().head(50)

In [45]:
surfprome = data.loc[(data.state_name == 'Capital Federal') & (data.surface_total_in_m2.notnull())].groupby(['place_name','property_type']).surface_total_in_m2.mean()
surfprome

place_name        property_type
Abasto            PH               139.846154
                  apartment         61.052632
                  house            334.333333
                  store            213.285714
Agronomía         PH                95.285714
                                      ...    
Villa Urquiza     store             93.631579
Villa del Parque  PH                92.659574
                  apartment         64.965015
                  house            221.382979
                  store            237.615385
Name: surface_total_in_m2, Length: 235, dtype: float64

In [46]:
'''completo lo superficie en base al barrio y tipo de las propiedades'''

data.loc[(data.state_name == 'Capital Federal') & (data.surface_total_in_m2.isnull()),'surface_total_in_m2'] = data.loc[(data.state_name == 'Capital Federal') & (data.surface_total_in_m2.isnull())].apply(lambda x: surfprome[(x['place_name'],x['property_type'])],axis=1)  

In [47]:
data.loc[(data.state_name == 'Capital Federal')].groupby(['place_name','property_type']).price_aprox_usd.describe()


count           mean            std  \
place_name       property_type                                        
Abasto           PH              13.0  176353.846154  119416.418577   
                 apartment       77.0  125282.206104   60950.952201   
                 house            3.0  470000.000000   43588.989435   
                 store            8.0  413125.000000  320055.101283   
Agronomía        PH               7.0  210714.285714   58696.150600   
...                               ...            ...            ...   
Villa Urquiza    store           19.0  225909.413158  188993.119711   
Villa del Parque PH              47.0  174131.914894   72038.166534   
                 apartment      344.0  151561.387064   73790.384769   
                 house           47.0  367468.085106  128444.176855   
                 store           13.0  377798.260000  237069.244239   

                                      min       25%        50%       75%  \
place_name       property_type                                             
Abasto           PH              55000.00   98700.0  160000.00  205000.0   
                 apartment       38000.00   83974.0  118000.00  150000.0   
                 house          420000.00  455000.0  490000.00  495000.0   
                 store          125000.00  280000.0  380000.00  385000.0   
Agronomía        PH             130000.00  175000.0  210000.00  237500.0   
...                                   ...       ...        ...       ...   
Villa Urquiza    store           48000.00   89500.0  143878.85  322500.0   
Villa del Parque PH              79900.00  115000.0  162000.00  224500.0   
                 apartment       59459.00   95000.0  140000.00  186237.5   
                 house          164000.00  277500.0  360000.00  455000.0   
                 store           22377.38  184000.0  330000.00  630000.0   

                                      max  
place_name       property_type             
Abasto           PH              510000.0  
                 apartment       440000.0  
                 house           500000.0  
                 store          1160000.0  
Agronomía        PH              310000.0  
...                                   ...  
Villa Urquiza    store           620000.0  
Villa del Parque PH              330000.0  
                 apartment       655000.0  
                 house           650000.0  
                 store           830000.0  

[235 rows x 8 columns]

In [48]:
data.loc[(data.state_name == 'Capital Federal')].groupby(['place_name','property_type']).surface_total_in_m2.describe()



count        mean         std    min    25%  \
place_name       property_type                                                
Abasto           PH              13.0  139.846154  145.487597   22.0   65.0   
                 apartment       77.0   61.052632   32.896526   24.0   38.0   
                 house            3.0  334.333333   70.401231  260.0  301.5   
                 store            8.0  213.285714  174.836016   44.0  116.5   
Agronomía        PH               7.0   95.285714   50.694228   20.0   74.0   
...                               ...         ...         ...    ...    ...   
Villa Urquiza    store           19.0   93.631579   80.599290   20.0   25.0   
Villa del Parque PH              47.0   92.659574   63.362064    1.0   53.0   
                 apartment      344.0   64.965015   39.664099    1.0   45.0   
                 house           47.0  221.382979  154.114237    9.0  105.0   
                 store           13.0  237.615385  197.389859   29.0   70.0   

                                  50%         75%    max  
place_name       property_type                            
Abasto           PH             120.0  160.000000  590.0  
                 apartment       55.0   71.000000  219.0  
                 house          343.0  371.500000  400.0  
                 store          199.0  204.071429  620.0  
Agronomía        PH              94.0  110.000000  185.0  
...                               ...         ...    ...  
Villa Urquiza    store           55.0  159.500000  243.0  
Villa del Parque PH              78.0  120.000000  254.0  
                 apartment       64.0   74.000000  561.0  
                 house          190.0  331.000000  552.0  
                 store          180.0  450.000000  561.0  

[235 rows x 8 columns]

In [156]:
data.loc[(data.state_name == 'Capital Federal')].place_name.unique()

array(['Mataderos', 'Liniers', 'Belgrano', 'Palermo Soho', 'Palermo',
       'Flores', 'Boedo', 'Las Cañitas', 'Puerto Madero', 'Balvanera',
       'Caballito', 'Nuñez', 'San Telmo', 'Almagro', 'Capital Federal',
       'Colegiales', 'Floresta', 'Barrio Norte', 'Barracas', 'Recoleta',
       'Congreso', 'Villa Crespo', 'Chacarita', 'Constitución',
       'Villa Urquiza', 'Palermo Hollywood', 'Saavedra', 'Monserrat',
       'Pompeya', 'Parque Chas', 'Paternal', 'Agronomía',
       'Villa Pueyrredón', 'Coghlan', 'Parque Centenario', 'Villa Luro',
       'Villa Devoto', 'Boca', 'Parque Avellaneda', 'San Cristobal',
       'Abasto', 'Versalles', 'Villa del Parque', 'Monte Castro',
       'Retiro', 'Parque Patricios', 'San Nicolás', 'Villa Santa Rita',
       'Palermo Chico', 'Centro / Microcentro', 'Tribunales', 'Once',
       'Parque Chacabuco', 'Velez Sarsfield', 'Catalinas',
       'Villa General Mitre', 'Palermo Viejo', 'Villa Lugano',
       'Villa Ortuzar', 'Villa Soldati', 'Villa Re